In [0]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import files
import io

import imghdr
import os

print(tf.__version__)

2.2.0-rc2


In [0]:
tf.executing_eagerly()

True

# Using Drive Mounting Instead (Tutorial: https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92)

To mount Google Drive:
- Run the code below
- Click the URL 
- Sign in & click Allow to get the authorization code
- Copy the authorization code and paste it to:
  Enter your authorization code: 
  ____________

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


After mounting, Collab will be connected to your Google Drive
Click the File button on the left <-, click 'drive' then 'My Drive', what we need is:
- sample_labels.csv file and the ct_scan_img.zip
- If it is not there, you can copy it from the Bangkit Group Project Folder
- Bangkit Group Project (Assig #5)/DATASET/CT_SCAN/ct_scan_img.zip"

Getting the sample_labels.csv in Colab:
- Find the sample_labels.csv file in your drive folder, right-click, and click COPY PATH
- Change the path variable below with the path you copied

In [0]:
path = "/content/drive/My Drive/FIONA CT Scan/sample_labels.csv"
sample_labels_all = pd.read_csv(path)

In [0]:
sample_labels_all.head()

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y
0,00000013_005.png,Emphysema|Infiltration|Pleural_Thickening|Pneu...,5,13,060Y,M,AP,3056,2544,0.139,0.139
1,00000013_026.png,Cardiomegaly|Emphysema,26,13,057Y,M,AP,2500,2048,0.168,0.168
2,00000017_001.png,No Finding,1,17,077Y,M,AP,2500,2048,0.168,0.168
3,00000030_001.png,Atelectasis,1,30,079Y,M,PA,2992,2991,0.143,0.143
4,00000032_001.png,Cardiomegaly|Edema|Effusion,1,32,055Y,F,AP,2500,2048,0.168,0.168


We want each label to have 1 finding, therefore for those with multiple findings, we are picking one out of them according to our defined ranking.

In [0]:
#sample_labels = sample_labels_all.sample(5000, replace=False) #[:1000]

In [0]:
ranking = ["pleural_thickening", "effusion", "consolidation", "hernia", "pneumonia", "fibrosis", 
           "edema", "emphysema", "cardiomegaly", "pneumothorax", "mass", "nodule", 
           "atelectasis", "infiltration", "no finding"]
#[14, 4, 5, 6, 7, 8, 1, 2, 9, 10, ]
#["pleural_thickening", "consolidation", "effusion", "pneumonia", ]

In [0]:
def pick_one(label):
  splitted = label.split('|')
  print(splitted)
  lowered = [s.lower() for s in splitted]
  print(lowered)
  index_rank = [ranking.index(word) for word in lowered]
  print(index_rank)
  min_rank = min(index_rank)
  print(min_rank)
  picked = ranking[min_rank]
  return picked
  #index_found = [options.index(word) for word in lowered if word in options]
  #result = index_found

  #return result

In [0]:
result = pick_one('Emphysema|Infiltration|Pleural_Thickening|Pneumothorax')
result

['Emphysema', 'Infiltration', 'Pleural_Thickening', 'Pneumothorax']
['emphysema', 'infiltration', 'pleural_thickening', 'pneumothorax']
[7, 13, 0, 9]
0


'pleural_thickening'

In [0]:
init_labels = sample_labels_all['Finding Labels'].tolist()
print(len(init_labels))
print(init_labels[:5])

5606
['Emphysema|Infiltration|Pleural_Thickening|Pneumothorax', 'Cardiomegaly|Emphysema', 'No Finding', 'Atelectasis', 'Cardiomegaly|Edema|Effusion']


In [0]:
picked_labels = [pick_one(label) for label in init_labels]

Streaming output truncated to the last 5000 lines.
['no finding']
[14]
14
['Edema', 'Effusion', 'Nodule']
['edema', 'effusion', 'nodule']
[6, 1, 11]
1
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Infiltration']
['infiltration']
[13]
13
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Atelectasis']
['atelectasis']
[12]
12
['No Finding']
['no finding']
[14]
14
['Infiltration']
['infiltration']
[13]
13
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Infiltration']
['infiltration']
[13]
13
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['No Finding']
['no finding']
[14]
14
['Atelectasis']
['atelectasis']
[12]
12
['Infiltration']
['infiltration']
[13]
13
['Pleural_Thickening']
['pleural_thickening']
[0]
0
['No Finding']
['no finding']
[14]
14
['No Finding']
['

In [0]:
picked_labels[:10]

['pleural_thickening',
 'emphysema',
 'no finding',
 'atelectasis',
 'effusion',
 'consolidation',
 'no finding',
 'no finding',
 'effusion',
 'no finding']

In [0]:
sample_labels_all['Old Labels'] = sample_labels_all['Finding Labels']
sample_labels_all['Finding Labels'] = picked_labels 

In [0]:
sample_labels_all.groupby('Finding Labels').count()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
Finding Labels,,,,,,,,,,,
atelectasis,249,249,249,249,249,249,249,249,249,249,249
cardiomegaly,66,66,66,66,66,66,66,66,66,66,66
consolidation,156,156,156,156,156,156,156,156,156,156,156
edema,76,76,76,76,76,76,76,76,76,76,76
effusion,606,606,606,606,606,606,606,606,606,606,606
emphysema,89,89,89,89,89,89,89,89,89,89,89
fibrosis,60,60,60,60,60,60,60,60,60,60,60
hernia,11,11,11,11,11,11,11,11,11,11,11
infiltration,503,503,503,503,503,503,503,503,503,503,503


In [0]:
average_num = sample_labels_all.groupby('Finding Labels').count()[['Image Index']].reset_index()

In [0]:
average_num

,Finding Labels,Image Index
0,atelectasis,249
1,cardiomegaly,66
2,consolidation,156
3,edema,76
4,effusion,606
5,emphysema,89
6,fibrosis,60
7,hernia,11
8,infiltration,503
9,mass,152


In [0]:
average_is = np.mean(average_num['Image Index'].tolist())
average_is

373.73333333333335

In [0]:
stad = np.std(average_num['Image Index'].tolist())
stad

731.4618665536996

### Balancing the Data

In [0]:
count = sample_labels_all.groupby('Finding Labels').count()[['Image Index']].reset_index()
count

,Finding Labels,Image Index
0,atelectasis,249
1,cardiomegaly,66
2,consolidation,156
3,edema,76
4,effusion,606
5,emphysema,89
6,fibrosis,60
7,hernia,11
8,infiltration,503
9,mass,152


In [0]:
above_100 = count[count['Image Index'] > 100]
above_100

,Finding Labels,Image Index
0,atelectasis,249
2,consolidation,156
4,effusion,606
8,infiltration,503
9,mass,152
10,no finding,3044
11,nodule,202
12,pleural_thickening,176
14,pneumothorax,172


In [0]:
above_100_without_nofinding = above_100[above_100['Finding Labels'] != 'no finding']
above_100_without_nofinding

,Finding Labels,Image Index
0,atelectasis,249
2,consolidation,156
4,effusion,606
8,infiltration,503
9,mass,152
11,nodule,202
12,pleural_thickening,176
14,pneumothorax,172


In [0]:
below_100 = count[count['Image Index'] < 100]
below_100

,Finding Labels,Image Index
1,cardiomegaly,66
3,edema,76
5,emphysema,89
6,fibrosis,60
7,hernia,11
13,pneumonia,44


In [0]:
sample_labels = sample_labels_all[sample_labels_all['Finding Labels'] != 'cardiomegaly']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'edema']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'emphysema']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'fibrosis']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'hernia']
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'pneumonia']

In [0]:
# TESTING WITH NO -> NO FINDINGS
print(len(sample_labels))
sample_labels = sample_labels[sample_labels['Finding Labels'] != 'no finding']
print(len(sample_labels))

5260
2216


In [0]:
# Add back SOME no findings
average_label_num = int(np.mean(above_100_without_nofinding['Image Index'].tolist()))
print(average_label_num)

277


In [0]:
sample_no_finding = sample_labels_all[sample_labels_all['Finding Labels'] == 'no finding'].sample(average_label_num, replace=False)

In [0]:
sample_no_finding

,Image Index,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
4479,00022312_001.png,no finding,1,22312,024Y,M,PA,2706,2865,0.143,0.143,No Finding
730,00003837_008.png,no finding,8,3837,052Y,F,PA,2500,2048,0.168,0.168,No Finding
60,00000334_005.png,no finding,5,334,069Y,F,PA,2670,2991,0.143,0.143,No Finding
543,00002798_001.png,no finding,1,2798,069Y,M,AP,2500,2048,0.171,0.171,No Finding
4829,00025387_002.png,no finding,2,25387,056Y,M,AP,3056,2544,0.139,0.139,No Finding
...,...,...,...,...,...,...,...,...,...,...,...,...
1049,00005478_000.png,no finding,0,5478,054Y,M,PA,2500,2048,0.171,0.171,No Finding
1964,00010294_008.png,no finding,8,10294,050Y,M,AP,2500,2048,0.168,0.168,No Finding
1412,00007328_012.png,no finding,12,7328,041Y,M,AP,2500,2048,0.171,0.171,No Finding
1005,00005236_000.png,no finding,0,5236,048Y,M,PA,3056,2544,0.139,0.139,No Finding


In [0]:
sample_labels_added = sample_labels.append(sample_no_finding)

In [0]:
print(len(sample_labels_added))
sample_labels_added.groupby('Finding Labels').count()

2493


,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
Finding Labels,,,,,,,,,,,
atelectasis,249,249,249,249,249,249,249,249,249,249,249
consolidation,156,156,156,156,156,156,156,156,156,156,156
effusion,606,606,606,606,606,606,606,606,606,606,606
infiltration,503,503,503,503,503,503,503,503,503,503,503
mass,152,152,152,152,152,152,152,152,152,152,152
no finding,277,277,277,277,277,277,277,277,277,277,277
nodule,202,202,202,202,202,202,202,202,202,202,202
pleural_thickening,176,176,176,176,176,176,176,176,176,176,176
pneumothorax,172,172,172,172,172,172,172,172,172,172,172


In [0]:
# TRY LARGER VALUES
sample_labels = sample_labels_added.sample(2480, replace=False) #500 TRY ALL cannot 2493

In [0]:
sample_labels.groupby('Finding Labels').count()

,Image Index,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImageWidth,OriginalImageHeight,OriginalImagePixelSpacing_x,OriginalImagePixelSpacing_y,Old Labels
Finding Labels,,,,,,,,,,,
atelectasis,248,248,248,248,248,248,248,248,248,248,248
consolidation,156,156,156,156,156,156,156,156,156,156,156
effusion,601,601,601,601,601,601,601,601,601,601,601
infiltration,501,501,501,501,501,501,501,501,501,501,501
mass,152,152,152,152,152,152,152,152,152,152,152
no finding,274,274,274,274,274,274,274,274,274,274,274
nodule,202,202,202,202,202,202,202,202,202,202,202
pleural_thickening,175,175,175,175,175,175,175,175,175,175,175
pneumothorax,171,171,171,171,171,171,171,171,171,171,171


In [0]:
labels = sample_labels['Finding Labels'].tolist()
len(labels)

2480

In [0]:
#above_100_labels = above_100['Finding Labels'].tolist()

In [0]:
#sample_labels = sample_labels_all[sample_labels_all['Finding Labels'] == above_100_labels[0]]

In [0]:
#[for label in above_100_labels]

Getting the CT Scan Images to Colab:
- Find the ct_scan_img.zip file in your drive folder (in Colab)
- Right-click, then click COPY PATH
- Paste the change the path I have, which is: "/content/drive/My Drive/Bangkit Group Project (Assig #5)/DATASET/CT_SCAN/ct_scan_img.zip" BELOW, with the one you have.
- And then run the !unzip to unzip the images

In [0]:
!unzip -uq "/content/drive/My Drive/FIONA CT Scan/ct_scan_img.zip"

From the Sample Labels dataframe / table, we get the filename of the images, and make a list out of that called 'image_dir' list.

In [0]:
image_dir = sample_labels['Image Index'].tolist()

In [0]:
# Just checking, carry on~
sample_labels[sample_labels['Image Index'] == image_dir[0]]['Finding Labels']

3452    effusion
Name: Finding Labels, dtype: object

In [0]:
type(image_dir[0])

str

In [0]:
image_dir[:5]

['00017036_020.png',
 '00016142_050.png',
 '00014130_001.png',
 '00015031_009.png',
 '00003393_020.png']

The RAM available in Google Colab is LIMITED, I believe initially it gives 15 GB.
If there is an error of: Colab doesn't have enough RAM, usually it asks if we want MORE RAM, click yes!
And it will give us 35 GB.

But, it still won't be enough for us to try with ALL of the images.
Therefore, we will only use 100 Images.

In [0]:
test_dir = image_dir #[:100] # Taking the first 100 Images
len(test_dir)

2480

In [0]:
test_dir[:5]

['00017036_020.png',
 '00016142_050.png',
 '00014130_001.png',
 '00015031_009.png',
 '00003393_020.png']

Creating the Datasets:
The tf.data.Dataset API supports writing descriptive and efficient input pipelines. Dataset usage follows a common pattern:

- Create a source dataset from your input data.
- Apply dataset transformations to preprocess the data.
- Iterate over the dataset and process the elements.

Iteration happens in a streaming fashion, so the full dataset does not need to fit into memory.

Sources can be from:
- LIST, using -> from_tensor_slices
- Files given filename, using -> list_files

In [0]:
dataset_from_list = tf.data.Dataset.list_files(test_dir)

In [0]:
dataset_from_list

<ShuffleDataset shapes: (), types: tf.string>

In [0]:
for element in dataset_from_list:
  print(element)

tf.Tensor(b'./00021637_001.png', shape=(), dtype=string)
tf.Tensor(b'./00025832_009.png', shape=(), dtype=string)
tf.Tensor(b'./00008463_001.png', shape=(), dtype=string)
tf.Tensor(b'./00001301_048.png', shape=(), dtype=string)
tf.Tensor(b'./00005801_002.png', shape=(), dtype=string)
tf.Tensor(b'./00020458_022.png', shape=(), dtype=string)
tf.Tensor(b'./00017691_010.png', shape=(), dtype=string)
tf.Tensor(b'./00013917_038.png', shape=(), dtype=string)
tf.Tensor(b'./00023009_003.png', shape=(), dtype=string)
tf.Tensor(b'./00006736_009.png', shape=(), dtype=string)
tf.Tensor(b'./00001006_002.png', shape=(), dtype=string)
tf.Tensor(b'./00026282_000.png', shape=(), dtype=string)
tf.Tensor(b'./00021255_004.png', shape=(), dtype=string)
tf.Tensor(b'./00018608_016.png', shape=(), dtype=string)
tf.Tensor(b'./00005029_003.png', shape=(), dtype=string)
tf.Tensor(b'./00017584_001.png', shape=(), dtype=string)
tf.Tensor(b'./00011925_010.png', shape=(), dtype=string)
tf.Tensor(b'./00022257_000.png'

In [0]:
dataset_of_filenames = tf.data.Dataset.from_tensor_slices(test_dir) 

In [0]:
dataset_of_filenames

<TensorSliceDataset shapes: (), types: tf.string>

In [0]:
for element in dataset_of_filenames:
  print(element)

tf.Tensor(b'00017036_020.png', shape=(), dtype=string)
tf.Tensor(b'00016142_050.png', shape=(), dtype=string)
tf.Tensor(b'00014130_001.png', shape=(), dtype=string)
tf.Tensor(b'00015031_009.png', shape=(), dtype=string)
tf.Tensor(b'00003393_020.png', shape=(), dtype=string)
tf.Tensor(b'00018356_005.png', shape=(), dtype=string)
tf.Tensor(b'00000099_003.png', shape=(), dtype=string)
tf.Tensor(b'00007735_041.png', shape=(), dtype=string)
tf.Tensor(b'00005502_015.png', shape=(), dtype=string)
tf.Tensor(b'00011719_022.png', shape=(), dtype=string)
tf.Tensor(b'00014473_000.png', shape=(), dtype=string)
tf.Tensor(b'00021610_002.png', shape=(), dtype=string)
tf.Tensor(b'00011959_005.png', shape=(), dtype=string)
tf.Tensor(b'00000084_000.png', shape=(), dtype=string)
tf.Tensor(b'00023085_006.png', shape=(), dtype=string)
tf.Tensor(b'00006821_005.png', shape=(), dtype=string)
tf.Tensor(b'00022816_000.png', shape=(), dtype=string)
tf.Tensor(b'00008734_001.png', shape=(), dtype=string)
tf.Tensor(

As we can see, whether it is made using list_files or from_tensor_slices it outputs the same dataset. 

In [0]:
for f in dataset_of_filenames.take(1):
    print(f.numpy())
    print(f)
    print(type(f))
    filename_str = str(f.numpy())[2:-1]
    print(filename_str)
    print(type(filename_str))
    print(type(str(f.numpy())))
    print(image_dir[0])
    print(filename_str == image_dir[0])
    print(sample_labels[sample_labels['Image Index'] == filename_str].reset_index().iloc[0]['Finding Labels'])


b'00017036_020.png'
tf.Tensor(b'00017036_020.png', shape=(), dtype=string)
<class 'tensorflow.python.framework.ops.EagerTensor'>
00017036_020.png
<class 'str'>
<class 'str'>
00017036_020.png
True
effusion


In [0]:
def parse_data_without_augmentation(filename, IMAGE_SIZE): #, scores, IMAGE_SIZE):
    '''
    Loads the image file without any augmentation. Used for validation set.
    Args:
        filename: the filename from the record
        scores: the scores from the record
    Returns:
        an image referred to by the filename and its scores
    '''

    image = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (IMAGE_SIZE, IMAGE_SIZE))
    image = (tf.cast(image, tf.float32) - 127.5) / 127.5

    return image #, label #, scores
    
""""
    print("FILENAME")
  
    print(str(filename))
    print(tf.strings.as_string(filename))
    tf.compat.v1.enable_eager_execution()
    filename_str = str(filename.numpy())[2:-1]
    # sample_labels['Finding Labels'].tolist()
    label = sample_labels[sample_labels['Image Index'] == filename_str].reset_index().iloc[0]['Finding Labels']
    #sample_labels[sample_labels['Image Index'] == str(filename)]['Finding Labels']
    print(label)

"""""


'"\n    print("FILENAME")\n  \n    print(str(filename))\n    print(tf.strings.as_string(filename))\n    tf.compat.v1.enable_eager_execution()\n    filename_str = str(filename.numpy())[2:-1]\n    # sample_labels[\'Finding Labels\'].tolist()\n    label = sample_labels[sample_labels[\'Image Index\'] == filename_str].reset_index().iloc[0][\'Finding Labels\']\n    #sample_labels[sample_labels[\'Image Index\'] == str(filename)][\'Finding Labels\']\n    print(label)\n\n'

In [0]:
IMG_WIDTH = 160
IMG_HEIGHT = 160
IMG_SIZE = 160
IMAGE_SIZE = 160
SHUFFLE_BUFFER_SIZE = 1000
BATCH_SIZE = 25 #1 #####32
#print("BATCH SIZE: "+str(BATCH_SIZE))
IMG_HEIGHT = 160 #224
IMG_WIDTH = 160 #224
IMG_SIZE = 160
image_count = len(test_dir)
print(image_count)
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
AUTOTUNE = tf.data.experimental.AUTOTUNE

2480


To actually get the Image, we need to parse every bit of the image from the image file, using the function:
parse_data_without_augmentation on the Dataset of filename we have -> list_ds.

num_parallel_calls: (Optional.) If specified, the implementation creates a threadpool, which is used to fetch inputs from cycle elements asynchronously and in parallel. The default behavior is to fetch inputs from cycle elements synchronously with no parallelism. If the value tf.data.experimental.AUTOTUNE is used, then the number of parallel calls is set dynamically based on available CPU.

In [0]:
image_dataset = dataset_of_filenames.map(lambda x: parse_data_without_augmentation(x, IMAGE_SIZE), num_parallel_calls=AUTOTUNE)

In [0]:
image_dataset

<ParallelMapDataset shapes: (160, 160, 3), types: tf.float32>

Some things we can do to Dataset object:
- Apply transformation
- Concatenate
- Enumerate (Create tuples of (index, element) with specified index starting point)
- Filter


Since we cannot index a TF Dataset object, we can turn the Dataset into List form using the code below

In [0]:
img_ds_list = list(image_dataset.as_numpy_iterator())

In [0]:
img_ds_list[0]

array([[[-0.9843137 , -0.9843137 , -0.9843137 ],
        [-0.9843137 , -0.9843137 , -0.9843137 ],
        [-0.9843137 , -0.9843137 , -0.9843137 ],
        ...,
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ]],

       [[-0.9843137 , -0.9843137 , -0.9843137 ],
        [-0.9843137 , -0.9843137 , -0.9843137 ],
        [-0.9768628 , -0.9768628 , -0.9768628 ],
        ...,
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ]],

       [[-0.9843137 , -0.9843137 , -0.9843137 ],
        [-0.98352945, -0.98352945, -0.98352945],
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        ...,
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ],
        [-0.9764706 , -0.9764706 , -0.9764706 ]],

       ...,

       [[-0.43333334, -0.43333334, -0.43333334],
        [-0

In [0]:
# The Images
for image in image_dataset:
  print(image)

Streaming output truncated to the last 5000 lines.
[[[-0.79607844 -0.79607844 -0.79607844]
  [-0.80368626 -0.80368626 -0.80368626]
  [-0.79882354 -0.79882354 -0.79882354]
  ...
  [-0.8039216  -0.8039216  -0.8039216 ]
  [-0.8039216  -0.8039216  -0.8039216 ]
  [-0.8000784  -0.8000784  -0.8000784 ]]

 [[-0.8039216  -0.8039216  -0.8039216 ]
  [-0.8109804  -0.8109804  -0.8109804 ]
  [-0.80823535 -0.80823535 -0.80823535]
  ...
  [-0.8117647  -0.8117647  -0.8117647 ]
  [-0.8117647  -0.8117647  -0.8117647 ]
  [-0.8112157  -0.8112157  -0.8112157 ]]

 [[-0.8117647  -0.8117647  -0.8117647 ]
  [-0.8117647  -0.8117647  -0.8117647 ]
  [-0.8117647  -0.8117647  -0.8117647 ]
  ...
  [-0.81764704 -0.81764704 -0.81764704]
  [-0.8117647  -0.8117647  -0.8117647 ]
  [-0.81411755 -0.81411755 -0.81411755]]

 ...

 [[-0.8643137  -0.8643137  -0.8643137 ]
  [-0.8666667  -0.8666667  -0.8666667 ]
  [-0.85490197 -0.85490197 -0.85490197]
  ...
  [-0.85882354 -0.85882354 -0.85882354]
  [-0.8666667  -0.8666667  -0.866

Splitting Dataset into Training, Testing & Validation 

In [0]:
total = len(sample_labels)
print("Total Data To Use: "+str(total))

Total Data To Use: 2480


In [0]:
# Note that we only have 100 images
train_size = int(0.7*total) #70
test_size = int(0.15*total) #15
val_size = int(0.15*total) #15
print(train_size)
print(test_size)
print(val_size)

1736
372
372


In [0]:
# Splitting Image Data
train_img = image_dataset.take(train_size)
test_img = image_dataset.skip(train_size)
val_img = test_img.skip(val_size)
test_img = test_img.take(test_size)

In [0]:
train_img

<TakeDataset shapes: (160, 160, 3), types: tf.float32>

In [0]:
train_lab = labels[0:train_size]
print(len(train_lab))
test_lab = labels[train_size:(train_size+test_size)]
print(len(test_lab))
val_lab = labels[(train_size+test_size):]
print(len(val_lab))

1736
372
372


In [0]:
for img in train_img:
  print(img)

Streaming output truncated to the last 5000 lines.
[[[ 0.21027449  0.21027449  0.21027449]
  [-0.46721575 -0.46721575 -0.46721575]
  [-0.9109804  -0.9109804  -0.9109804 ]
  ...
  [-0.9529412  -0.9529412  -0.9529412 ]
  [-0.9529412  -0.9529412  -0.9529412 ]
  [-0.9529412  -0.9529412  -0.9529412 ]]

 [[-0.00494115 -0.00494115 -0.00494115]
  [-0.6392157  -0.6392157  -0.6392157 ]
  [-0.95686275 -0.95686275 -0.95686275]
  ...
  [-0.9529412  -0.9529412  -0.9529412 ]
  [-0.9529412  -0.9529412  -0.9529412 ]
  [-0.9529412  -0.9529412  -0.9529412 ]]

 [[-0.03254903 -0.03254903 -0.03254903]
  [-0.6462745  -0.6462745  -0.6462745 ]
  [-0.9627451  -0.9627451  -0.9627451 ]
  ...
  [-0.9529412  -0.9529412  -0.9529412 ]
  [-0.9529412  -0.9529412  -0.9529412 ]
  [-0.9529412  -0.9529412  -0.9529412 ]]

 ...

 [[ 0.15333323  0.15333323  0.15333323]
  [-0.734902   -0.734902   -0.734902  ]
  [-1.         -1.         -1.        ]
  ...
  [-0.9529412  -0.9529412  -0.9529412 ]
  [-0.9529412  -0.9529412  -0.952

In [0]:
def show_batch(image_batch, num_elements): #, label_batch):
  plt.figure(figsize=(10,10))
  for n in range(num_elements):
      ax = plt.subplot(5,5,n+1)
      plt.imshow(image_batch[n])
      #plt.title(CLASS_NAMES[label_batch[n]==1][0].title())
      plt.axis('off')

In [0]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  # This is a small dataset, only load it once, and keep it in memory.
  # use `.cache(filename)` to cache preprocessing work for datasets that don't
  # fit in memory.
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()

  #ds = ds.shuffle(buffer_size=shuffle_buffer_size)

  # Repeat forever
  #ds = ds.repeat()

  ds = ds.batch(BATCH_SIZE)

  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=AUTOTUNE)

  return ds

In [0]:
BATCH_SIZE = train_size
print("Train size: "+str(BATCH_SIZE))
train_img_batch = prepare_for_training(train_img)
BATCH_SIZE = test_size
print("Test size: "+str(BATCH_SIZE))
test_img_batch = prepare_for_training(test_img)
BATCH_SIZE = val_size
print("Val size: "+str(BATCH_SIZE))
val_img_batch = prepare_for_training(val_img)

Train size: 1736
Test size: 372
Val size: 372


In [0]:
#for img in train_img:
#  print(img)

In [0]:
#for img in train_img_batch:
#  print(img)

In [0]:
train_img_batch

<PrefetchDataset shapes: (None, 160, 160, 3), types: tf.float32>

In [0]:
train_batch = next(iter(train_img_batch))
test_batch = next(iter(test_img_batch))
val_batch = next(iter(val_img_batch))

In [0]:
print(train_batch.shape)
print(test_batch.shape)
print(val_batch.shape)

(1736, 160, 160, 3)
(372, 160, 160, 3)
(372, 160, 160, 3)


In [0]:
#show_batch(val_batch.numpy(), val_size)

## Feature Extraction Using Pre-trained MobileNetV2 Model

Often, it is better to get features using a pre-trained embedding model that have been trained with millions of images, instead of training our own embedding. We will use MobileNetV2.

In [0]:
IMG_SHAPE = (IMG_SIZE, IMG_SIZE, 3)

In [0]:
print(IMG_SIZE)

160


In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

9412608/9406464 [==============================] - 0s 0us/step


Extracting features for train, test, and validation.

In [0]:
train_features = base_model(train_batch)
test_features = base_model(test_batch)
val_features = base_model(val_batch)

In [0]:
print(train_features.shape)
print(test_features.shape)
print(val_features.shape)

(1736, 5, 5, 1280)
(372, 5, 5, 1280)
(372, 5, 5, 1280)


We need to convert the numpy version of these features, to Input Tensors, before inputting it to the model.

In [0]:
# fnp = features numpy
train_fnp = train_features.numpy()
test_fnp = test_features.numpy()
val_fnp = val_features.numpy()

In [0]:
train_input = tf.convert_to_tensor(train_fnp, dtype=tf.float32)
test_input = tf.convert_to_tensor(test_fnp, dtype=tf.float32)
val_input = tf.convert_to_tensor(val_fnp, dtype=tf.float32)

In [0]:
test_input.shape

TensorShape([372, 5, 5, 1280])

Another Way:

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [0]:
train_batch_ave = global_average_layer(train_features)
print(train_batch_ave.shape)

(1736, 1280)


In [0]:
test_batch_ave = global_average_layer(test_features)
print(test_batch_ave.shape)

(372, 1280)


In [0]:
val_batch_ave = global_average_layer(val_features)
print(val_batch_ave.shape)

(372, 1280)


In [0]:
prediction_layer = tf.keras.layers.Dense(256) #(2) #256

In [0]:
train_pred_batch = prediction_layer(train_batch_ave)
print(train_pred_batch.shape)

(1736, 256)


In [0]:
test_pred_batch = prediction_layer(test_batch_ave)
print(test_pred_batch.shape)

(372, 256)


In [0]:
val_pred_batch = prediction_layer(val_batch_ave)
print(val_pred_batch.shape)

(372, 256)


In [0]:
train_input = train_pred_batch.numpy()

In [0]:
train_input.shape

(1736, 256)

In [0]:
test_input = test_pred_batch.numpy()
test_input.shape

(372, 256)

In [0]:
val_input = val_pred_batch.numpy()
val_input.shape

(372, 256)

# Pre-processing the Labels:


Since the labels are texts, we need to create an embedding or number / vector representations from it. 

We will use:
- LabelEncoder -> assigning integer 0 to number of unique categorical variable to each categorical variable. 
- OneHotEncoding

In [0]:
num_of_labels = len(labels)
print(num_of_labels)

2480


In [0]:
unique_labels = sample_labels['Finding Labels'].unique().tolist()
print(unique_labels[:5])
num_of_unique = len(unique_labels)
num_of_unique

['effusion', 'atelectasis', 'infiltration', 'nodule', 'consolidation']


9

In [0]:
labels[:5]

['effusion', 'effusion', 'atelectasis', 'effusion', 'infiltration']

In [0]:
from sklearn import preprocessing

Creating the Label Encoder object from ALL of our Labels

In [0]:
le = preprocessing.LabelEncoder()

In [0]:
le.fit(labels)

LabelEncoder()

Encode train, test, and validation labels with Integer Values.

In [0]:
train_label_value = le.transform(train_lab)
test_label_value = le.transform(test_lab)
val_label_value = le.transform(val_lab)

In [0]:
len(train_lab)

1736

In [0]:
label_to_value = pd.DataFrame(data=train_lab[:10], columns=['Original Label'])
label_to_value['Label Value'] = train_label_value[:10]

In [0]:
all_labels_value = le.transform(labels)
len(all_labels_value)

2480

Below is the Example of Encoding Validation Label to their label values. 

Notice that all 'No Finding' becomes the value 24

In [0]:
label_to_value

,Original Label,Label Value
0,effusion,2
1,effusion,2
2,atelectasis,0
3,effusion,2
4,infiltration,3
5,effusion,2
6,effusion,2
7,effusion,2
8,nodule,6
9,infiltration,3


In [0]:
train_one_hot = tf.one_hot(train_label_value, num_of_unique, on_value=1, off_value=0)
test_one_hot = tf.one_hot(test_label_value, num_of_unique, on_value=1, off_value=0)
val_one_hot = tf.one_hot(val_label_value, num_of_unique, on_value=1, off_value=0)

In [0]:
all_labels_one_hot = tf.one_hot(all_labels_value, num_of_unique, on_value=1, off_value=0)

In [0]:
label_to_value['One Hot'] = train_one_hot[:10]

In [0]:
label_to_value

,Original Label,Label Value,One Hot
0,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
1,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
2,atelectasis,0,"(tf.Tensor(1, shape=(), dtype=int32), tf.Tenso..."
3,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
4,infiltration,3,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
5,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
6,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
7,effusion,2,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
8,nodule,6,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."
9,infiltration,3,"(tf.Tensor(0, shape=(), dtype=int32), tf.Tenso..."


In [0]:
all_labels_df = pd.DataFrame(data=labels, columns=['Label'])
all_labels_df['Value'] = all_labels_value
all_labels_df['One Hot'] = all_labels_one_hot

In [0]:
all_labels_df

# Creating the CT Scan Image Classification Model

In [0]:
from tensorflow.keras.models import Sequential

Additional Metrics Functions on top of Accuracy

In [0]:
def recall_m(y_true, y_pred):
        true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+tf.keras.backend.epsilon()))

Creating the Model TRIAL #1:

In [0]:
model_1 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_of_unique, activation='softmax')
])

In [0]:
model_1.summary()

In [0]:
model_1.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

In [0]:
history = model_1.fit(train_input, train_one_hot, epochs=25, validation_data = (val_input, val_one_hot))

In [0]:
model_2 = keras.Sequential([
    keras.Input(shape=(256)),     
    keras.layers.Dense(500, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.45)),
    keras.layers.Dense(320, activation='relu',kernel_regularizer=tf.keras.regularizers.l2(0.11)),     
    keras.layers.Dense(300,kernel_regularizer=tf.keras.regularizers.l2(0.07)),     
    keras.layers.Dropout(rate=0.001),
    keras.layers.Dense(180),    
    #keras.layers.Dense(10, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.07)),                 

    keras.layers.Dense(num_of_unique, activation='softmax')
])

model_2.summary()

model_2.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), metrics=['accuracy', precision_m, recall_m, f1_m])

#history = model_2.fit(train_input, train_one_hot, epochs=25, validation_data = (val_input, val_one_hot)) # epochs=300

In [0]:
# Using Early Stopping:
callback = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=2)

In [0]:
# Using Early Stopping:
callback_2 = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=20)

In [0]:
history = model_2.fit(train_input, train_one_hot, epochs=25, callbacks=[callback_2], validation_data = (val_input, val_one_hot)) # epochs=300

In [0]:
model_3 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax'),
])

model_3.summary()
model_3.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

In [0]:
history_3 = model_3.fit(train_input, train_one_hot, epochs=25, validation_data = (val_input, val_one_hot))

In [0]:
model_4 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax'),
])

model_4.summary()
model_4.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

In [0]:
history_4 = model_4.fit(train_input, train_one_hot, epochs=300, callbacks=[callback], validation_data = (val_input, val_one_hot))

In [0]:
model_5 = keras.Sequential([
    keras.Input(shape=(256)), 
    keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.25)),
    keras.layers.Dropout(rate=0.7),
    keras.layers.Dense(num_of_unique, activation='softmax'),
])

model_5.summary()
model_5.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy', precision_m, recall_m, f1_m])

In [0]:
history_5 = model_5.fit(train_input, train_one_hot, epochs=300, validation_data = (val_input, val_one_hot))

In [0]:
#history = model_1.fit(train_input, train_one_hot, epochs=20, validation_data = (val_input, val_one_hot))

EVALUATION

In [0]:
evaluation = model_1.evaluate(test_input, test_one_hot)

In [0]:
def create_evaluation_table(model, x_input, actual_label):
  test_pred = model_1.predict(x_input)
  argmarx_ = [np.argmax(pred) for pred in test_pred]
  pred_labels = [all_labels_df[all_labels_df.Value == val].reset_index().iloc[0]['Label'] for val in argmarx_]
  compare = pd.DataFrame(data=actual_label, columns=['Actual'])
  compare['Predicted'] = pred_labels
  return compare

In [0]:
test_eval_table = create_evaluation_table(model_1, test_input, test_lab)

In [0]:
#test_eval_table

In [0]:
#test_eval_table[0:50]

In [0]:
evaluation_2 = model_2.evaluate(test_input, test_one_hot)

In [0]:
evaluation_2_table = create_evaluation_table(model_2, val_input, val_lab)

In [0]:
#evaluation_2_table[0:60]

In [0]:
evaluation_3 = model_3.evaluate(test_input, test_one_hot)

In [0]:
evaluation_4 = model_4.evaluate(test_input, test_one_hot)

In [0]:
evaluation_5 = model_5.evaluate(test_input, test_one_hot)

In [0]:
val_eval = model_1.evaluate(val_input, val_one_hot)

In [0]:
val_eval_table = create_evaluation_table(model_1, val_input, val_lab)

In [0]:
val_eval_table

In [0]:
train_eval = model_1.evaluate(train_input, train_one_hot)

In [0]:
train_eval_table = create_evaluation_table(model_1, train_input, train_lab)

In [0]:
train_eval_table